In [4]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.cross_validation import cross_val_score, train_test_split

In [5]:
data = pd.read_csv('/Users/caixy/sandbox/kaggle-kb/data.csv')

In [20]:
#to caculate the sum of different action type such as jump shot or running jump shot
total_actions = dict(data.action_type.value_counts())

threshold = 10
data['type'] = data.apply(lambda row: row['action_type'] if total_actions[row['action_type']] >= threshold \
                          else row['combined_shot_type'], axis = 1)

data['time_remaining'] = data.apply(lambda row: row['minutes_remaining']*60 + row['seconds_remaining'], axis = 1)

# TODO: tune this parameter
threshold = 3
# TODO: find out why he cant hit @ 14 secs to go
anomaly = 14